In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import re
from nltk import ngrams
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from ckiptagger import WS

ws = WS('./data')

/Users/jackson/.pyenv/versions/3.8.3/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/Users/jackson/.pyenv/versions/3.8.3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [3]:
df = pd.read_csv('./processed/chinese_categories.csv')

In [4]:
df.head()

,Id,C1,C2,C3,C4,C5,C6,C7
0,469,保健與美容,NaN,NaN,NaN,NaN,NaN,NaN
1,2915,保健與美容,個人保養,NaN,NaN,NaN,NaN,NaN
2,6921,保健與美容,個人保養,仿曬噴霧,NaN,NaN,NaN,NaN
3,528,保健與美容,個人保養,刮鬍修容,NaN,NaN,NaN,NaN
4,2681,保健與美容,個人保養,刮鬍修容,刮鬍刷,NaN,NaN,NaN


In [5]:
df = df.drop(columns = 'Id')

In [6]:
df.head(50)

,C1,C2,C3,C4,C5,C6,C7
0,保健與美容,NaN,NaN,NaN,NaN,NaN,NaN
1,保健與美容,個人保養,NaN,NaN,NaN,NaN,NaN
2,保健與美容,個人保養,仿曬噴霧,NaN,NaN,NaN,NaN
3,保健與美容,個人保養,刮鬍修容,NaN,NaN,NaN,NaN
4,保健與美容,個人保養,刮鬍修容,刮鬍刷,NaN,NaN,NaN
5,保健與美容,個人保養,刮鬍修容,刮鬍碗/杯,NaN,NaN,NaN
6,保健與美容,個人保養,刮鬍修容,刮鬍組,NaN,NaN,NaN
7,保健與美容,個人保養,刮鬍修容,刮鬍膏,NaN,NaN,NaN
8,保健與美容,個人保養,刮鬍修容,手動刮鬍刀和刀片,NaN,NaN,NaN
9,保健與美容,個人保養,刮鬍修容,止血筆,NaN,NaN,NaN


In [7]:
all_words = defaultdict(set)
for idx, row in enumerate(df.values):
    for r in row:
        if r == r:
            all_words[df['C1'][idx]].add(r)

In [8]:
all_words

defaultdict(set,
            {'保健與美容': {'乳液/乳霜',
              '乳液/防曬乳塗抹用具',
              '乾洗手/濕紙巾',
              '乾耳器',
              '人體彩繪顏料/粉底',
              '仿曬乳',
              '仿曬噴霧',
              '便盆',
              '保健與美容',
              '保冷劑',
              '保險套',
              '修皮剪',
              '修錶工具組',
              '個人保養',
              '假牙',
              '假牙修補組',
              '假牙清潔產品',
              '假牙黏著劑',
              '假睫毛',
              '假睫毛卸除液',
              '假睫毛輔助夾',
              '假睫毛配件',
              '假睫毛黏著劑',
              '健康照護',
              '健身與營養',
              '健髮產品',
              '光學治療用燈',
              '全營養食物泥',
              '冷熱療法',
              '凡士林',
              '刮舌器',
              '刮鬍修容',
              '刮鬍刷',
              '刮鬍碗/杯',
              '刮鬍組',
              '刮鬍膏',
              '剪髮刀',
              '助曬劑',
              '助曬產品',
              '助行器',
              '化妝品',
              '化妝工具清潔液',
              '化妝水/收斂水',
       

In [9]:
for key in all_words.keys():
    all_words[key].add(key)

In [22]:
cat_keywords = {}
for key in all_words.keys():
    words = list(all_words[key])
    tok = ws(words, segment_delimiter_set = {'?', '？', '!', '！', '。', ',',   
                                   '，', ';', ':', '/'})
    tokens = []
    for t in tok:
        tokens += t
    cv = CountVectorizer(max_features = 10)
    try:
        word_count_vector = cv.fit_transform(tokens)
    except ValueError:                            #no product in this category e.g. media
        cat_keywords[key] = []
    cat_keywords[key] = [word.replace(' ', '') for word in cv.vocabulary_.keys()]

In [23]:
cat_keywords

{'保健與美容': ['配件', '清潔', '用品', '治療', '工具組', '保養', '工具', '產品', '噴霧', '營養'],
 '動物/寵物補給品': ['飼料', '配件', '爬蟲', '兩棲', '水族箱', '寵物', '動物', '鳥類', '用品', '鳥籠'],
 '商業與各產業': ['工作', '安全', '標誌', '看板', '顯微鏡', '實驗室', '配件', '醫療', '免洗', '刺青'],
 '媒體': ['模型', '玩具', '手冊', '用品', '使用', '器材', 'dvd', '樂譜', '產品', '木工'],
 '嬰幼兒': ['嬰兒', '玩具', '嬰幼兒', '配件', '奶瓶', '奶嘴', '背巾', '安全', '安撫', '尿布'],
 '宗教和慶典': ['走道', '宗教', '用品', '禱文卡', '面紗', '紀念', '儀式', '花童', '祭壇', '禱告'],
 '家具': ['隔間', '配件', '廚房', '戶外', '沙發', '飯廳', '嬰兒', '家具組', '長凳', '家具'],
 '居家和庭園': ['廚房', '園藝', '割草機', '配件', '戶外', '用品', '食物', '裝飾', '泳池', '清潔劑'],
 '成人用品': ['槍枝', '清潔', '情趣', '用品', '成人', '武器', '情色', '彈藥', '裝備', '噴霧'],
 '服飾與配件': ['戲服', '配件', '服裝', '嬰幼兒', '手套', '外套', '制服', '內褲', '頭飾', '服飾'],
 '玩具和遊戲': ['遊戲組', '積木', '遊戲', '玩具', '配件', '遙控', '陀螺', '玩具組', '娃娃', '拼圖'],
 '相機和光學元件': ['相機', '配件', '三腳架', '攝影機', '望遠鏡', '閃光燈', '光學', '暗房', '攝影棚', '鏡頭'],
 '硬體': ['水管', '配件', '馬桶', '幫浦', '工具', '水龍頭', '油漆', '管線', '用品', '櫥櫃'],
 '藝術/娛樂': ['配件', '樂器', '工具', '手工藝', '銅管', '零件', '薩克

In [24]:
with open('./processed/cat_keywords.pickle', 'wb') as out:
    pickle.dump(cat_keywords, out)